# Pénzérme felismerő algoritmus hatásfokának ellenőrzése

In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt


class Picture_of_coin:
    count_5 = 0
    count_10 = 0
    count_20 = 0
    count_50 = 0
    count_100 = 0
    count_200 = 0
    found_coins =0
    def __init__(self,count_5,count_10,count_20,count_50,count_100,count_200,found_coins):
        self.count_5 = count_5
        self.count_10 = count_10
        self.count_20 = count_20
        self.count_50 = count_50
        self.count_100 = count_100
        self.count_200 = count_200
        self.found_coins = found_coins

    def sum_value(self):
        return (self.count_5 * 5) + (self.count_10*10) + (self.count_20 * 20) + (self.count_50 * 50) + (self.count_100 * 100) + (self.count_200 *200)

    def set_found_coins(self, n):
        self.found_coins = n

    def print(self):
        print(str(self.count_5) + " db. 5 forintos, " + str(self.count_10) + " db. 10 forintos, " + str(self.count_20) + " db. 20 forintos, " + str(self.count_50) + " db. 50 forintos, " + str(self.count_100) + " db. 100 forintos, " + str(self.count_200) + " db. 200 forintos, összesen " + str(self.found_coins) + " db érme.")


f = open(os.path.join("proba_kepek", "eredmenyek.txt"))
x = f.readlines()
f.close()

found_result_array = []
result_array = []

for i in x:
    y =(i.split())
    z = Picture_of_coin(int(y[3]),int(y[4]),int(y[5]),int(y[6]),int(y[7]),int(y[8]),y[1])
    result_array.append(z)

for i in range(1, (len(result_array))):
    filename = str(i) + ".jpg"
    img = cv2.imread(os.path.join("proba_kepek", filename))
    d = 1024 / img.shape[1]
    dim = (1024, int(img.shape[0] * d))
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    img_gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    blur_img = cv2.GaussianBlur(img_gray, (21, 21), 0)
    circles = cv2.HoughCircles(blur_img, cv2.HOUGH_GRADIENT, 1, 120, param1=40, param2=35, minRadius=50, maxRadius=160)

    circles_color = []
    for i in circles[0, :]:
        x = int(i[0])
        y = int(i[1])
        d = int(i[2])
        roi = resized[y - d:y + d, x - d:x + d]

        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        avg_hsv_per_row = np.average(hsv, axis=0)
        avg_hsv = np.average(avg_hsv_per_row, axis=0)

        if (avg_hsv[2] > 140):
            hsv[:, :, 2] -= 40
            img2 = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
            avg_color_per_row = np.average(img2, axis=0)
            avg_color = np.average(avg_color_per_row, axis=0)
            circles_color.append(avg_color)

        else:
            avg_color_per_row = np.average(roi, axis=0)
            avg_color = np.average(avg_color_per_row, axis=0)
            circles_color.append(avg_color)


    z = 0

    actual_pic = Picture_of_coin(0, 0, 0, 0, 0, 0,0)

    for i in circles[0, :]:
        if (i[2] <= 130 and circles_color[z][0] < 85):
            actual_pic.count_5 += 1
        elif (i[2] > 130 and i[2] <= 136 and circles_color[z][0] < 85):
            actual_pic.count_100 +=1
        elif (i[2] > 144 and i[2] <= 153 and circles_color[z][0] < 85):
            actual_pic.count_20 += 1
        elif (i[2] > 153 and circles_color[z][0] < 85):
            actual_pic.count_200 +=1
        elif (i[2] > 135 and i[2] <= 150):
            actual_pic.count_10 +=1
        elif (i[2] > 150 and i[2] <= 160):
            actual_pic.count_50 += 1

        z = z + 1
    actual_pic.set_found_coins(circles.shape[1])
    found_result_array.append(actual_pic)

correct_coin_numbers = 0
correct_coin_values = 0

for i in range(0, (len(result_array) - 1)):


    if (int(result_array[i].found_coins) == int(found_result_array[i].found_coins)):
        correct_coin_numbers += 1
    if (int(result_array[i].sum_value()) == int(found_result_array[i].sum_value())):
        correct_coin_values += 1

print(" Az érmek darab számát", (len(result_array)) ,"kép feldolgozása során, az algoritmus ", int(correct_coin_numbers/(len(result_array)-2) * 100), "% ban találta el, míg az érmék összértékét ", int(correct_coin_values/(len(result_array)-2) * 100), "% ban. \n" )
print("A hibák részletezése: \n")
for i in range(0, (len(result_array) - 1)):
    if (int(result_array[i].found_coins) != int(found_result_array[i].found_coins)) or (int(result_array[i].sum_value()) != int(found_result_array[i].sum_value())) :
        print(str(i+1) + ". képen hibásan ismerte fel az érméket az algoritmus. \n")
        print("A képen látgató érmék száma:")
        result_array[i].print()
        print("Az algoritmus által megtalált érmék száma:")
        found_result_array[i].print()
        print("\n")
        


 Az érmek darab számát 27 kép feldolgozása során, az algoritmus  96 % ban találta el, míg az érmék összértékét  80 % ban. 

A hibák részletezése: 

7. képen hibásan ismerte fel az érméket az algoritmus. 

A képen látgató érmék száma:
1 db. 5 forintos, 1 db. 10 forintos, 1 db. 20 forintos, 1 db. 50 forintos, 1 db. 100 forintos, 1 db. 200 forintos, összesen 6 db érme.
Az algoritmus által megtalált érmék száma:
1 db. 5 forintos, 1 db. 10 forintos, 2 db. 20 forintos, 1 db. 50 forintos, 1 db. 100 forintos, 0 db. 200 forintos, összesen 6 db érme.


9. képen hibásan ismerte fel az érméket az algoritmus. 

A képen látgató érmék száma:
1 db. 5 forintos, 1 db. 10 forintos, 1 db. 20 forintos, 1 db. 50 forintos, 1 db. 100 forintos, 1 db. 200 forintos, összesen 6 db érme.
Az algoritmus által megtalált érmék száma:
1 db. 5 forintos, 1 db. 10 forintos, 1 db. 20 forintos, 1 db. 50 forintos, 1 db. 100 forintos, 1 db. 200 forintos, összesen 7 db érme.


10. képen hibásan ismerte fel az érméket az algori